# Calculator Load and Check
+ Load a calculator template
+ Load in dummy values
+ Check the outputs are as expected

## Objects
### Calculation template
+ Spreadsheet template with all the relationships between fields expressed. Libreoffice is used to execute this, excel can also be used.

### Measurement Module
+ Someway to take the names of fields and translate these into measurement commands. In this case the names match a modbus lookup table.

### Calculation Output
+ Filled in calculation template with all fields calculated

### Test matrix
+ Defines the names of the fields to check
+ Has the operation which is in a lookup table. We're only using simple operations (min/max or equal) more complex relationships should be expressed in the calculation template

In [1]:
import pandas
import pyexcel, openpyxl

In [2]:
template = "./calculator-sheet-blank-template.xlsx"
workbook = openpyxl.open("./calculator-sheet-blank-template.xlsx")
df = pandas.read_excel(template)
sheet = workbook.active
columns_names = {}
for column, in workbook.active.iter_cols(min_row=1, max_row=1, values_only=False):
    columns_names[column.value] = column.column - 1#_letter

In [3]:
measured_rows = df[df["type"] == "measured"]
print(measured_rows)

                name       value unit      type
0            version         NaN  NaN  measured
1   firmware_version         NaN  NaN  measured
2       compile_time         NaN  NaN  measured
3       compile_date         NaN  NaN  measured
4      slave_address         NaN  NaN  measured
5   p3_3_micro_volts         NaN   uV  measured
6       p3_3_reading         NaN  ADC  measured
11   p23_micro_volts  23000000.0   uV  measured
12       p23_reading      1200.0  ADC  measured
17    p5_micro_volts         NaN   uV  measured
18        p5_reading      2500.0  ADC  measured
23      fault_status         NaN  NaN  measured


In [4]:
dummy_values_df = pandas.read_excel("./dummy-values.xlsx")

In [5]:
for i, line in measured_rows.iterrows():
    coordinate = workbook.active.cell(row=i+2, column=columns_names["value"]+1).coordinate
    value = dummy_values_df[dummy_values_df["name"] == line["name"]].iloc[0]["value"]
    workbook.active[coordinate] = value
    print(coordinate, workbook.active[coordinate].value)

B2 0.8.X
B3 0.1.8
B4 09:09:00
B5 September 9th, 2023
B6 246
B7 3260000
B8 2024
B13 23000000
B14 1200
B19 5000000
B20 2500
B25 9


In [6]:
workbook.save("/tmp/out.xlsx")

In [21]:
import os, sys, importlib
sys.path.append("../")
import pyetest
importlib.reload(pyetest)
out_file = "/tmp/out-export.xlsx"
pyetest.reload_spreadsheet("/tmp/out.xlsx", out_file)
# Theres a special condition where libreoffice will just exit if the output file is the same as the input. Converting to xls isnt required if the output directory is different.
# os.system("cd /tmp && libreoffice --calc --convert-to xls /tmp/out.xlsx && libreoffice --calc --convert-to xlsx /tmp/out.xls")

0

In [22]:
processed_df = pandas.read_excel(out_file)

In [23]:
values_matrix_df = pandas.read_excel("./value-limits.xlsx").dropna(subset=["operation"])
def in_range(value, line):
    return value >= line["min"] and value <= line["max"]

def equals(value, line):
    return value == line["expected value"]
    
operations = {"<=>": in_range, "==": equals}

In [24]:
values = []
checks = []

for i, value_line in values_matrix_df.iterrows():
    line = processed_df[processed_df["name"] == value_line["name"]].iloc[0]
    value = line["value"]
    values.append(value)
    checks.append(operations[value_line["operation"]](value, value_line))

values_matrix_df["value"] = values
values_matrix_df["checks"] = checks

In [25]:
print(values_matrix_df)

                 name       expected value unit         min         max  \
0             version                0.8.X  NaN         NaN         NaN   
1    firmware_version                0.1.8  NaN         NaN         NaN   
2        compile_time             09:09:00  NaN         NaN         NaN   
3        compile_date  September 9th, 2023  NaN         NaN         NaN   
4       slave_address                  246  NaN         NaN         NaN   
5    p3_3_micro_volts              3300000   uV   3200000.0   3400000.0   
7           +3.3ERROR                    0    %         0.0         3.0   
8     p23_micro_volts             23000000   uV  18000000.0  30000000.0   
10           +23ERROR                    0    %         0.0         3.0   
11     p5_micro_volts              5000000   uV   4900000.0   5100000.0   
13            +5ERROR                    0    %         0.0         3.0   
15           +5 Fault                    0  NaN         NaN         NaN   
16          +24 Fault    

In [13]:
tests_passed = all(values_matrix_df["checks"])
print(tests_passed)

False
